# Time Series Cross Validation Experiments

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

from torch.utils.data import DataLoader, TensorDataset

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
import os

In [9]:
# Load datasets
pwd = os.getcwd()

mi_data = pd.read_csv(pwd + '\merged_data.csv')
mbw_data = pd.read_csv(fr'C:\Users\ivane\Desktop\Dissertation\data\merged_data.csv')

# Convert date columns to datetime
mi_data['date'] = pd.to_datetime(mi_data['date'])
mbw_data['Date'] = pd.to_datetime(mbw_data['Date'])

# Sort by date
mi_data = mi_data.sort_values('date')
mbw_data = mbw_data.sort_values('Date')

# Reset index
mi_data = mi_data.reset_index(drop=True)
mbw_data = mbw_data.reset_index(drop=True)

print("Malta Independent Data Shape:", mi_data.shape)
print("Malta Business Weekly Data Shape:", mbw_data.shape)

Malta Independent Data Shape: (1001, 6)
Malta Business Weekly Data Shape: (430, 6)


# Malta Independent Experiments

## LSTM (without sentiment variables)

In [12]:
# Create sequences for LSTM
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length - 1):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length, -1]  # Target is the last column (Close price)
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(mi_data[['Open', 'Close', 'Change']])

# Create sequences
seq_length = 10
X, y = create_sequences(scaled_data, seq_length)

# Initialize time series cross validation
tscv = TimeSeriesSplit(n_splits=5)

# Store results for each fold
fold_results = []

# Perform cross validation
for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Build and compile model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        verbose=1
    )
    
    # Make predictions
    y_pred = model.predict(X_val)
    
    # Calculate metrics
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    fold_results.append({
        'fold': fold + 1,
        'mae': mae,
        'r2': r2
    })
    
    print(f'Fold {fold + 1} Results:')
    print(f'MAE: {mae:.4f}')
    print(f'R2 Score: {r2:.4f}')

# Print average results
avg_results = pd.DataFrame(fold_results).mean()
print('\nAverage Results:')
print(f'Average MAE: {avg_results["mae"]:.4f}')
print(f'Average R2 Score: {avg_results["r2"]:.4f}')


Fold 1
Epoch 1/50
6/6 [==============================] - 4s 154ms/step - loss: 0.0766 - val_loss: 0.0276
Epoch 2/50
6/6 [==============================] - 0s 16ms/step - loss: 0.0227 - val_loss: 0.0274
Epoch 3/50
6/6 [==============================] - 0s 15ms/step - loss: 0.0052 - val_loss: 0.0428
Epoch 4/50
6/6 [==============================] - 0s 15ms/step - loss: 0.0084 - val_loss: 0.0407
Epoch 5/50
6/6 [==============================] - 0s 15ms/step - loss: 0.0044 - val_loss: 0.0303
Epoch 6/50
6/6 [==============================] - 0s 14ms/step - loss: 0.0037 - val_loss: 0.0277
Epoch 7/50
6/6 [==============================] - 0s 15ms/step - loss: 0.0036 - val_loss: 0.0308
Epoch 8/50
6/6 [==============================] - 0s 18ms/step - loss: 0.0029 - val_loss: 0.0328
Epoch 9/50
6/6 [==============================] - 0s 15ms/step - loss: 0.0031 - val_loss: 0.0310
Epoch 10/50
6/6 [==============================] - 0s 16ms/step - loss: 0.0029 - val_loss: 0.0300
Epoch 11/50
6/6 [===

## LSTM (with sentiment variables)

In [13]:
# Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(mi_data[['Open', 'Close', 'Change', 'majority_sentiment', 'average_sentiment']])

# Create sequences
seq_length = 10
X, y = create_sequences(scaled_data, seq_length)

# Initialize time series cross validation
tscv = TimeSeriesSplit(n_splits=5)

# Store results for each fold
fold_results = []

# Perform cross validation
for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Build and compile model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        verbose=1
    )
    
    # Make predictions
    y_pred = model.predict(X_val)
    
    # Calculate metrics
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    fold_results.append({
        'fold': fold + 1,
        'mae': mae,
        'r2': r2
    })
    
    print(f'Fold {fold + 1} Results:')
    print(f'MAE: {mae:.4f}')
    print(f'R2 Score: {r2:.4f}')

# Print average results
avg_results = pd.DataFrame(fold_results).mean()
print('\nAverage Results:')
print(f'Average MAE: {avg_results["mae"]:.4f}')
print(f'Average R2 Score: {avg_results["r2"]:.4f}')


Fold 1
Epoch 1/50
6/6 [==============================] - 5s 166ms/step - loss: 0.2739 - val_loss: 0.0792
Epoch 2/50
6/6 [==============================] - 0s 16ms/step - loss: 0.0814 - val_loss: 0.0668
Epoch 3/50
6/6 [==============================] - 0s 16ms/step - loss: 0.0449 - val_loss: 0.0991
Epoch 4/50
6/6 [==============================] - 0s 15ms/step - loss: 0.0456 - val_loss: 0.1154
Epoch 5/50
6/6 [==============================] - 0s 16ms/step - loss: 0.0468 - val_loss: 0.0916
Epoch 6/50
6/6 [==============================] - 0s 18ms/step - loss: 0.0355 - val_loss: 0.0703
Epoch 7/50
6/6 [==============================] - 0s 15ms/step - loss: 0.0360 - val_loss: 0.0700
Epoch 8/50
6/6 [==============================] - 0s 15ms/step - loss: 0.0343 - val_loss: 0.0779
Epoch 9/50
6/6 [==============================] - 0s 15ms/step - loss: 0.0349 - val_loss: 0.0817
Epoch 10/50
6/6 [==============================] - 0s 16ms/step - loss: 0.0343 - val_loss: 0.0715
Epoch 11/50
6/6 [===

## CNN-LSTM (without sentiment variables)

In [14]:
# Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(mi_data[['Open', 'Close', 'Change']])

# Create sequences
seq_length = 10
X, y = create_sequences(scaled_data, seq_length)

# Initialize time series cross validation
tscv = TimeSeriesSplit(n_splits=5)

# Store results for each fold
fold_results = []

# Perform cross validation
for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Build and compile model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        verbose=1
    )
    
    # Make predictions
    y_pred = model.predict(X_val)
    
    # Calculate metrics
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    fold_results.append({
        'fold': fold + 1,
        'mae': mae,
        'r2': r2
    })
    
    print(f'Fold {fold + 1} Results:')
    print(f'MAE: {mae:.4f}')
    print(f'R2 Score: {r2:.4f}')

# Print average results
avg_results = pd.DataFrame(fold_results).mean()
print('\nAverage Results:')
print(f'Average MAE: {avg_results["mae"]:.4f}')
print(f'Average R2 Score: {avg_results["r2"]:.4f}')


Fold 1
Epoch 1/50
6/6 [==============================] - 5s 172ms/step - loss: 0.0769 - val_loss: 0.0302
Epoch 2/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0111 - val_loss: 0.0190
Epoch 3/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0075 - val_loss: 0.0266
Epoch 4/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0059 - val_loss: 0.0312
Epoch 5/50
6/6 [==============================] - 0s 11ms/step - loss: 0.0052 - val_loss: 0.0227
Epoch 6/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0033 - val_loss: 0.0192
Epoch 7/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0034 - val_loss: 0.0219
Epoch 8/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0029 - val_loss: 0.0239
Epoch 9/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0031 - val_loss: 0.0217
Epoch 10/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0028 - val_loss: 0.0202
Epoch 11/50
6/6 [===

## CNN-LSTM (with sentiment variables)

In [15]:
# Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(mi_data[['Open', 'Close', 'Change', 'majority_sentiment', 'average_sentiment']])

# Create sequences
seq_length = 10
X, y = create_sequences(scaled_data, seq_length)

# Initialize time series cross validation
tscv = TimeSeriesSplit(n_splits=5)

# Store results for each fold
fold_results = []

# Perform cross validation
for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Build and compile model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        verbose=1
    )
    
    # Make predictions
    y_pred = model.predict(X_val)
    
    # Calculate metrics
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    fold_results.append({
        'fold': fold + 1,
        'mae': mae,
        'r2': r2
    })
    
    print(f'Fold {fold + 1} Results:')
    print(f'MAE: {mae:.4f}')
    print(f'R2 Score: {r2:.4f}')

# Print average results
avg_results = pd.DataFrame(fold_results).mean()
print('\nAverage Results:')
print(f'Average MAE: {avg_results["mae"]:.4f}')
print(f'Average R2 Score: {avg_results["r2"]:.4f}')


Fold 1
Epoch 1/50
6/6 [==============================] - 5s 155ms/step - loss: 0.4956 - val_loss: 0.2749
Epoch 2/50
6/6 [==============================] - 0s 11ms/step - loss: 0.1355 - val_loss: 0.0585
Epoch 3/50
6/6 [==============================] - 0s 11ms/step - loss: 0.0808 - val_loss: 0.0584
Epoch 4/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0447 - val_loss: 0.0777
Epoch 5/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0484 - val_loss: 0.0878
Epoch 6/50
6/6 [==============================] - 0s 11ms/step - loss: 0.0445 - val_loss: 0.0623
Epoch 7/50
6/6 [==============================] - 0s 11ms/step - loss: 0.0334 - val_loss: 0.0528
Epoch 8/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0362 - val_loss: 0.0530
Epoch 9/50
6/6 [==============================] - 0s 12ms/step - loss: 0.0339 - val_loss: 0.0584
Epoch 10/50
6/6 [==============================] - 0s 13ms/step - loss: 0.0341 - val_loss: 0.0662
Epoch 11/50
6/6 [===

# Malta Business Weekly Experiments

## LSTM (without sentiment variables)

In [16]:
# Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(mbw_data[['Open', 'Close', 'Change']])

# Create sequences
seq_length = 10
X, y = create_sequences(scaled_data, seq_length)

# Initialize time series cross validation
tscv = TimeSeriesSplit(n_splits=5)

# Store results for each fold
fold_results = []

# Perform cross validation
for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Build and compile model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        verbose=1
    )
    
    # Make predictions
    y_pred = model.predict(X_val)
    
    # Calculate metrics
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    fold_results.append({
        'fold': fold + 1,
        'mae': mae,
        'r2': r2
    })
    
    print(f'Fold {fold + 1} Results:')
    print(f'MAE: {mae:.4f}')
    print(f'R2 Score: {r2:.4f}')

# Print average results
avg_results = pd.DataFrame(fold_results).mean()
print('\nAverage Results:')
print(f'Average MAE: {avg_results["mae"]:.4f}')
print(f'Average R2 Score: {avg_results["r2"]:.4f}')


Fold 1
Epoch 1/50
3/3 [==============================] - 4s 363ms/step - loss: 0.0823 - val_loss: 0.0585
Epoch 2/50
3/3 [==============================] - 0s 26ms/step - loss: 0.0207 - val_loss: 0.0280
Epoch 3/50
3/3 [==============================] - 0s 23ms/step - loss: 0.0375 - val_loss: 0.0298
Epoch 4/50
3/3 [==============================] - 0s 24ms/step - loss: 0.0237 - val_loss: 0.0419
Epoch 5/50
3/3 [==============================] - 0s 24ms/step - loss: 0.0192 - val_loss: 0.0540
Epoch 6/50
3/3 [==============================] - 0s 24ms/step - loss: 0.0237 - val_loss: 0.0557
Epoch 7/50
3/3 [==============================] - 0s 23ms/step - loss: 0.0242 - val_loss: 0.0485
Epoch 8/50
3/3 [==============================] - 0s 23ms/step - loss: 0.0201 - val_loss: 0.0382
Epoch 9/50
3/3 [==============================] - 0s 23ms/step - loss: 0.0178 - val_loss: 0.0307
Epoch 10/50
3/3 [==============================] - 0s 23ms/step - loss: 0.0183 - val_loss: 0.0278
Epoch 11/50
3/3 [===

## LSTM (with sentiment variables)

In [17]:
# Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(mbw_data[['Open', 'Close', 'Change', 'sentiment', 'sentiment_score']])

# Create sequences
seq_length = 10
X, y = create_sequences(scaled_data, seq_length)

# Initialize time series cross validation
tscv = TimeSeriesSplit(n_splits=5)

# Store results for each fold
fold_results = []

# Perform cross validation
for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Build and compile model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        verbose=1
    )
    
    # Make predictions
    y_pred = model.predict(X_val)
    
    # Calculate metrics
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    fold_results.append({
        'fold': fold + 1,
        'mae': mae,
        'r2': r2
    })
    
    print(f'Fold {fold + 1} Results:')
    print(f'MAE: {mae:.4f}')
    print(f'R2 Score: {r2:.4f}')

# Print average results
avg_results = pd.DataFrame(fold_results).mean()
print('\nAverage Results:')
print(f'Average MAE: {avg_results["mae"]:.4f}')
print(f'Average R2 Score: {avg_results["r2"]:.4f}')


Fold 1
Epoch 1/50
3/3 [==============================] - 4s 371ms/step - loss: 0.1513 - val_loss: 0.0699
Epoch 2/50
3/3 [==============================] - 0s 23ms/step - loss: 0.0357 - val_loss: 0.0567
Epoch 3/50
3/3 [==============================] - 0s 23ms/step - loss: 0.0717 - val_loss: 0.0501
Epoch 4/50
3/3 [==============================] - 0s 23ms/step - loss: 0.0382 - val_loss: 0.0597
Epoch 5/50
3/3 [==============================] - 0s 22ms/step - loss: 0.0339 - val_loss: 0.0785
Epoch 6/50
3/3 [==============================] - 0s 22ms/step - loss: 0.0437 - val_loss: 0.0785
Epoch 7/50
3/3 [==============================] - 0s 24ms/step - loss: 0.0418 - val_loss: 0.0666
Epoch 8/50
3/3 [==============================] - 0s 23ms/step - loss: 0.0345 - val_loss: 0.0544
Epoch 9/50
3/3 [==============================] - 0s 33ms/step - loss: 0.0319 - val_loss: 0.0506
Epoch 10/50
3/3 [==============================] - 0s 22ms/step - loss: 0.0338 - val_loss: 0.0509
Epoch 11/50
3/3 [===

## CNN-LSTM (without sentiment variables)

In [18]:
# Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(mbw_data[['Open', 'Close', 'Change']])

# Create sequences
seq_length = 10
X, y = create_sequences(scaled_data, seq_length)

# Initialize time series cross validation
tscv = TimeSeriesSplit(n_splits=5)

# Store results for each fold
fold_results = []

# Perform cross validation
for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Build and compile model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        verbose=1
    )
    
    # Make predictions
    y_pred = model.predict(X_val)
    
    # Calculate metrics
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    fold_results.append({
        'fold': fold + 1,
        'mae': mae,
        'r2': r2
    })
    
    print(f'Fold {fold + 1} Results:')
    print(f'MAE: {mae:.4f}')
    print(f'R2 Score: {r2:.4f}')

# Print average results
avg_results = pd.DataFrame(fold_results).mean()
print('\nAverage Results:')
print(f'Average MAE: {avg_results["mae"]:.4f}')
print(f'Average R2 Score: {avg_results["r2"]:.4f}')


Fold 1
Epoch 1/50
3/3 [==============================] - 4s 423ms/step - loss: 0.1031 - val_loss: 0.0848
Epoch 2/50
3/3 [==============================] - 0s 22ms/step - loss: 0.0406 - val_loss: 0.0406
Epoch 3/50
3/3 [==============================] - 0s 21ms/step - loss: 0.0193 - val_loss: 0.0245
Epoch 4/50
3/3 [==============================] - 0s 24ms/step - loss: 0.0327 - val_loss: 0.0250
Epoch 5/50
3/3 [==============================] - 0s 27ms/step - loss: 0.0247 - val_loss: 0.0322
Epoch 6/50
3/3 [==============================] - 0s 23ms/step - loss: 0.0175 - val_loss: 0.0405
Epoch 7/50
3/3 [==============================] - 0s 27ms/step - loss: 0.0189 - val_loss: 0.0437
Epoch 8/50
3/3 [==============================] - 0s 23ms/step - loss: 0.0203 - val_loss: 0.0416
Epoch 9/50
3/3 [==============================] - 0s 20ms/step - loss: 0.0190 - val_loss: 0.0357
Epoch 10/50
3/3 [==============================] - 0s 20ms/step - loss: 0.0172 - val_loss: 0.0309
Epoch 11/50
3/3 [===

## CNN-LSTM (with sentiment variables)

In [19]:
# Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(mbw_data[['Open', 'Close', 'Change', 'sentiment', 'sentiment_score']])

# Create sequences
seq_length = 10
X, y = create_sequences(scaled_data, seq_length)

# Initialize time series cross validation
tscv = TimeSeriesSplit(n_splits=5)

# Store results for each fold
fold_results = []

# Perform cross validation
for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Build and compile model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        verbose=1
    )
    
    # Make predictions
    y_pred = model.predict(X_val)
    
    # Calculate metrics
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    fold_results.append({
        'fold': fold + 1,
        'mae': mae,
        'r2': r2
    })
    
    print(f'Fold {fold + 1} Results:')
    print(f'MAE: {mae:.4f}')
    print(f'R2 Score: {r2:.4f}')

# Print average results
avg_results = pd.DataFrame(fold_results).mean()
print('\nAverage Results:')
print(f'Average MAE: {avg_results["mae"]:.4f}')
print(f'Average R2 Score: {avg_results["r2"]:.4f}')


Fold 1
Epoch 1/50
3/3 [==============================] - 4s 381ms/step - loss: 0.4505 - val_loss: 0.3160
Epoch 2/50
3/3 [==============================] - 0s 21ms/step - loss: 0.2542 - val_loss: 0.1786
Epoch 3/50
3/3 [==============================] - 0s 20ms/step - loss: 0.1069 - val_loss: 0.0768
Epoch 4/50
3/3 [==============================] - 0s 20ms/step - loss: 0.0410 - val_loss: 0.0529
Epoch 5/50
3/3 [==============================] - 0s 32ms/step - loss: 0.0599 - val_loss: 0.0559
Epoch 6/50
3/3 [==============================] - 0s 19ms/step - loss: 0.0573 - val_loss: 0.0508
Epoch 7/50
3/3 [==============================] - 0s 21ms/step - loss: 0.0355 - val_loss: 0.0593
Epoch 8/50
3/3 [==============================] - 0s 23ms/step - loss: 0.0343 - val_loss: 0.0696
Epoch 9/50
3/3 [==============================] - 0s 21ms/step - loss: 0.0385 - val_loss: 0.0700
Epoch 10/50
3/3 [==============================] - 0s 19ms/step - loss: 0.0372 - val_loss: 0.0626
Epoch 11/50
3/3 [===

# CNN


In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Function to create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length, -1])  # Assuming the target variable is the last column
    return np.array(X), np.array(y)

# Load your datasets (replace with actual data)
mi_data = pd.read_csv(pwd + '\merged_data.csv')
mbw_data = pd.read_csv(fr'C:\Users\ivane\Desktop\Dissertation\data\merged_data.csv')

# Convert date columns to datetime
mi_data['date'] = pd.to_datetime(mi_data['date'])
mbw_data['Date'] = pd.to_datetime(mbw_data['Date'])

# Sort by date
mi_data = mi_data.sort_values('date')
mbw_data = mbw_data.sort_values('Date')

# Reset index
mi_data = mi_data.reset_index(drop=True)
mbw_data = mbw_data.reset_index(drop=True)

print("Malta Independent Data Shape:", mi_data.shape)
print("Malta Business Weekly Data Shape:", mbw_data.shape)

# Define a function to train and evaluate the CNN model
def train_cnn(data, features, title):
    print(f"\n### {title} ###")
    
    # Scale the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[features])
    
    # Create sequences
    seq_length = 10
    X, y = create_sequences(scaled_data, seq_length)
    
    # Initialize time series cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    
    # Store results for each fold
    fold_results = []
    
    # Perform cross-validation
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        print(f'\nFold {fold + 1}')
        
        # Split data
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Build and compile model
        model = Sequential([
            Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
            MaxPooling1D(pool_size=2),
            Conv1D(filters=32, kernel_size=3, activation='relu'),
            MaxPooling1D(pool_size=2),
            Flatten(),
            Dense(50, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        
        # Train model
        history = model.fit(
            X_train, y_train,
            epochs=50,
            batch_size=32,
            validation_data=(X_val, y_val),
            verbose=1
        )
        
        # Make predictions
        y_pred = model.predict(X_val)
        
        # Calculate metrics
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        
        fold_results.append({'fold': fold + 1, 'mae': mae, 'r2': r2})
        
        print(f'Fold {fold + 1} Results:')
        print(f'MAE: {mae:.4f}')
        print(f'R2 Score: {r2:.4f}')
    
    # Print average results
    avg_results = pd.DataFrame(fold_results).mean()
    print('\nAverage Results:')
    print(f'Average MAE: {avg_results["mae"]:.4f}')
    print(f'Average R2 Score: {avg_results["r2"]:.4f}')

# Train CNN models for Malta Independent (MI)
train_cnn(mi_data, ['Open', 'Close', 'Change'], "CNN (MI - Without Sentiment)")
train_cnn(mi_data, ['Open', 'Close', 'Change', 'majority_sentiment', 'average_sentiment'], "CNN (MI - With Sentiment)")

# Train CNN models for Malta Business Weekly (MBW)
train_cnn(mbw_data, ['Open', 'Close', 'Change'], "CNN (MBW - Without Sentiment)")
train_cnn(mbw_data, ['Open', 'Close', 'Change', 'sentiment', 'sentiment_score'], "CNN (MBW - With Sentiment)")


Malta Independent Data Shape: (1001, 6)
Malta Business Weekly Data Shape: (430, 6)

### CNN (MI - Without Sentiment) ###

Fold 1
Epoch 1/50
6/6 [==============================] - 1s 29ms/step - loss: 0.0061 - val_loss: 0.0364
Epoch 2/50
6/6 [==============================] - 0s 8ms/step - loss: 0.0037 - val_loss: 0.0325
Epoch 3/50
6/6 [==============================] - 0s 8ms/step - loss: 0.0031 - val_loss: 0.0316
Epoch 4/50
6/6 [==============================] - 0s 8ms/step - loss: 0.0029 - val_loss: 0.0313
Epoch 5/50
6/6 [==============================] - 0s 10ms/step - loss: 0.0030 - val_loss: 0.0301
Epoch 6/50
6/6 [==============================] - 0s 8ms/step - loss: 0.0029 - val_loss: 0.0297
Epoch 7/50
6/6 [==============================] - 0s 8ms/step - loss: 0.0030 - val_loss: 0.0284
Epoch 8/50
6/6 [==============================] - 0s 8ms/step - loss: 0.0029 - val_loss: 0.0304
Epoch 9/50
6/6 [==============================] - 0s 8ms/step - loss: 0.0029 - val_loss: 0.0264
Epoch